In [19]:
import csv
import subprocess
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
class Experiment:
    def __init__(self, parameters_file):
        self.parameters_file = parameters_file
        self.parameters=[]
        self.list_function_id=[]
        self.list_name=[]
        self.list_range_low=[]
        self.list_range_high=[]
        self.list_dimensions_vector=[]
        self.readParameters()
        self.solution_file = 'solutions.csv'
        self.solution_df = None
        
    def readParameters(self):
        with open(self.parameters_file) as csvfile:
            read_csv = csv.reader(csvfile, delimiter=',')
            for row in read_csv:
                self.parameters.append(row)
        
        #decode the parameters
        for i in range(1,len(self.parameters)):
            self.list_function_id.append(self.parameters[i][0]) #function_id
            self.list_name.append(self.parameters[i][1]) # function name
            self.list_range_low.append(self.parameters[i][2]) # range low
            self.list_range_high.append(self.parameters[i][3]) # range high
            self.list_dimensions_vector.append(self.parameters[i][4:]) # dimension vector, from 4 to the last element
            
    def delete_all_output_files(self):
        files = glob.glob('f*.csv')
        for f in files:
            os.remove(f)
            
    def merge_all_output_files(self):
        files = glob.glob("./f*.csv")
        #print(files)
        solutions=[]
        for f in files:
            with open(f) as csvfile:
                read_csv = csv.reader(csvfile, delimiter=',')
                for row in read_csv:
                    solutions.append(row)
        #print(solutions)
        
        
        with open(self.solution_file, mode='w') as solution_file:
            writer = csv.writer(solution_file, delimiter=',', lineterminator='\n',quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow(['function_id'
                             ,'n_samples'
                             ,'dimensions'
                             ,'range_low'
                             ,'range_high'
                             ,'mean'
                             ,'std_dev'
                             ,'range'
                             ,'median'
                             ,'time_ms'
                             ,'range_min'
                             ,'range_max'
                            ])
            writer.writerows(solutions)
            
    def plot(self):
        df=pd.read_csv('solutions.csv',index_col=False)
        df.sort_values(by=['dimensions', 'function_id'], inplace = True)
        self.solution_df = df
        
        #-----------plot running time
        fig, ax = plt.subplots(figsize=(8,6))
        for label, data in df.groupby('dimensions'):
            data.plot(kind='line',x='function_id',y='time_ms',ax=ax, style='o--', xticks=data['function_id'], label='dimension: %d'%label)
        plt.ylabel('time [ms]')    
        plt.legend()
        plt.grid()
        plt.title('Running time')
        
        #---------plot statistics
        for id, data in df.groupby('function_id'):
            fig, ax = plt.subplots(figsize=(8,4))
            plt.errorbar(data['dimensions'],data['mean'],yerr=data['std_dev'],fmt='o:', markersize=3, capsize=5, label='mean and std_dev')
            plt.plot(data['dimensions'],data['median'],label='median')
            plt.fill_between(data['dimensions'], data['range_min'], data['range_max'], color='gray', alpha=0.2, label='range')
            plt.xticks(data['dimensions'])
            plt.title("f%d : %s"%(id,test.list_name[id-1]))
            plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
            plt.xlabel('dimensions')
            plt.ylabel('solution')
            plt.grid()
        

    def run(self, n_samples):
        
        #delete all temp files
        self.delete_all_output_files()
        
        process_list = []
        for i in range(len(self.list_function_id)): # run each function
            for dimension in self.list_dimensions_vector[i]: # run different dimensions of each function
                #subprocess.Popen() can be multi process, subprocess.run() need to wait one process to finish
                process = subprocess.Popen([
                                "../bin/debug/project.exe"
                                #"../bin/release/project.o"
                                ,str(1) # optimization algorithm id
                                ,"../config/ga_config.csv"  # optimization configuration file
                                ,"f"+self.list_function_id[i]+".csv" # optimization result output file
                                ,self.list_function_id[i] #function_id
                                ,str(n_samples)
                                ,str(dimension)
                                ,self.list_range_low[i]
                                ,self.list_range_high[i]
                               ])
                process_list.append(process)
        
        # wait every process to finish
        for p in process_list:
            p.wait()
        print("----finished----")
        
        #self.merge_all_output_files()
        #delete all temp files
        #self.delete_all_output_files()

In [21]:
test = Experiment('../config/experiments.txt') 

In [22]:
test.run(200) # n random samples

----finished----


In [9]:
test.plot()

FileNotFoundError: [Errno 2] File b'solutions.csv' does not exist: b'solutions.csv'

In [8]:
test.solution_df